In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82164,139227,https://chinai.substack.com/p/chinai-168-aroun...,ChinAI #168: Around the Horn (edition 6),\N,\N,2022-01-09 15:39:25.727587+00,2022-01-09 15:39:25.727587+00,NaN,\N,ChinAI,73.0,0.0,\N
82165,139228,https://www.slowboring.com/p/sunday-thread-2b3,Sunday Thread,\N,www.slowboring.com,2022-01-09 15:39:25.730597+00,2022-01-09 15:40:50.171616+00,0.0,\N,Yglesias,74.0,0.0,\N
82166,139229,https://frenchpress.thedispatch.com/p/a-nation...,A Nation of Christians Is Not Necessarily a Ch...,\N,\N,2022-01-09 15:39:25.73287+00,2022-01-09 15:39:25.73287+00,NaN,\N,Dispatch,6.0,0.0,\N
82167,139230,https://www.lesswrong.com/posts/puYfAEJJomeode...,An Observation of Vavilov Day,\N,\N,2022-01-09 15:39:25.734913+00,2022-01-09 15:39:25.734913+00,NaN,\N,LW,62.0,0.0,\N


In [2]:
links['added'].max()

'2022-01-09 15:39:25.739325+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
82159    139220
82160    139223
82161    139224
82162    139225
82163    139226
Name: id, Length: 5800, dtype: int64

In [4]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [5]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 100 == 0 or i == len(new_links):
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/13


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82177,139240,https://mobile.twitter.com/mpershan/status/148...,https://mobile.twitter.com/mpershan/status/148...,\N,\N,2022-01-09 21:36:48.216996,2022-01-09 21:36:48.217010,\N,\N,Custom,20.0,0.0,NaN
82178,139241,https://mobile.twitter.com/Noahpinion/status/1...,https://mobile.twitter.com/Noahpinion/status/1...,\N,\N,2022-01-09 21:36:48.250140,2022-01-09 21:36:48.250153,\N,\N,Custom,29.0,0.0,NaN
82179,139242,https://mobile.twitter.com/messages/1414875069...,https://mobile.twitter.com/messages/1414875069...,\N,\N,2022-01-09 21:36:48.284504,2022-01-09 21:36:48.284516,\N,\N,Custom,79.0,0.0,NaN
82180,139243,https://mobile.twitter.com/BeijingToBrit/statu...,https://mobile.twitter.com/BeijingToBrit/statu...,\N,\N,2022-01-09 21:36:48.316594,2022-01-09 21:36:48.316607,\N,\N,Custom,81.0,0.0,NaN


In [6]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82177,139240,https://mobile.twitter.com/mpershan/status/148...,https://mobile.twitter.com/mpershan/status/148...,\N,\N,2022-01-09 21:36:48.216996,2022-01-09 21:36:48.217010,\N,\N,Custom,20.0,0.0,NaN
82178,139241,https://mobile.twitter.com/Noahpinion/status/1...,https://mobile.twitter.com/Noahpinion/status/1...,\N,\N,2022-01-09 21:36:48.250140,2022-01-09 21:36:48.250153,\N,\N,Custom,29.0,0.0,NaN
82179,139242,https://mobile.twitter.com/messages/1414875069...,https://mobile.twitter.com/messages/1414875069...,\N,\N,2022-01-09 21:36:48.284504,2022-01-09 21:36:48.284516,\N,\N,Custom,79.0,0.0,NaN
82180,139243,https://mobile.twitter.com/BeijingToBrit/statu...,https://mobile.twitter.com/BeijingToBrit/statu...,\N,\N,2022-01-09 21:36:48.316594,2022-01-09 21:36:48.316607,\N,\N,Custom,81.0,0.0,NaN


In [7]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82177,139240,https://mobile.twitter.com/mpershan/status/148...,https://mobile.twitter.com/mpershan/status/148...,\N,\N,2022-01-09 21:36:48.216996,2022-01-09 21:36:48.217010,\N,\N,Custom,20,0,NaN
82178,139241,https://mobile.twitter.com/Noahpinion/status/1...,https://mobile.twitter.com/Noahpinion/status/1...,\N,\N,2022-01-09 21:36:48.250140,2022-01-09 21:36:48.250153,\N,\N,Custom,29,0,NaN
82179,139242,https://mobile.twitter.com/messages/1414875069...,https://mobile.twitter.com/messages/1414875069...,\N,\N,2022-01-09 21:36:48.284504,2022-01-09 21:36:48.284516,\N,\N,Custom,79,0,NaN
82180,139243,https://mobile.twitter.com/BeijingToBrit/statu...,https://mobile.twitter.com/BeijingToBrit/statu...,\N,\N,2022-01-09 21:36:48.316594,2022-01-09 21:36:48.316607,\N,\N,Custom,81,0,NaN


In [8]:
links.to_csv('data/export.csv', index=False)